# Desafío 01

* Importación de librerías
* astype()

In [ ]:
#!pip install unidecode


In [1]:
import pandas as pd
import numpy as np
import re as re
import unidecode

In [2]:
# Para ver el contenido total de las columnas

pd.set_option('display.max_colwidth', -1)
#pd.set_option("display.max_columns", len(properatti.columns))

In [3]:
df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))


#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [ ]:
df.info(verbose=None, null_counts=True)

In [4]:
df.sample(1)['description']

32103    excelente departamento de 4 ambientes con dependencia, 3 banos, detalles de categoria, - pisos de madera(roble de eslavonia)- aire acondicionado central para toda la unidad (frio calor)- cortinas de enrollar electricas (las grandes)- hogar a lenos (funciona perfecto)- todos los muros pintados con satinol- parrilla en patio andaluz- agua caliente por termotanque en zona dormitorios y por calefon en sector cocina/lavadero en el corazon del parque centenario con una vista magnifica hacia el mismo, digno de ver
Name: description, dtype: object

* Información de las columnas

In [ ]:
# Busqueda de NaNs
df[pd.isnull(df['description'])]


### Reemplazo de NaN's
Esto se realiza porque la función de búsqueda explotaba 

In [5]:
df_a1=df.replace(np.nan,'sin-dato')
df_a1.head(2)

,Unnamed: 0,Unnamed: 0.1,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,0,sell,PH,mataderos,|argentina|capital federal|mataderos|,Argentina,capital federal,3.43079e+06,"-34.6618237,-58.5088387",...,40,1127.27,1550,sin-dato,sin-dato,sin-dato,http://www.properati.com.ar/15bo8_venta_ph_mataderos_lavadero_patio_inmobiliaria-fedele,"2 ambientes tipo casa planta baja por pasillo, refaccionado a nuevo, patio grande, con lavadero, living comedor con cocina integrada, artefacto de cocina, alacena, bano completo, y dormitorio. todo en excelente estado, para habitar.no es apto credito aviso publicado por pixel inmobiliario (servicio de paginas web para inmobiliarias).",2 amb tipo casa sin expensas en pb,https://thumbs4.properati.com/8/BluUYiHJLhgIIKufMJMwHtquzhk=/trim/198x0/smart/filters:strip_icc()/inmobiliariafedele.com/photos/CARHUE%202278-IMG_3400.JPG
1,1,1,sell,apartment,la plata,|argentina|bs.as. g.b.a. zona sur|la plata|,Argentina,bs.as. g.b.a. zona sur,3.43204e+06,"-34.9038831,-57.9643295",...,sin-dato,sin-dato,sin-dato,sin-dato,sin-dato,sin-dato,http://www.properati.com.ar/15bob_venta_departamentos_la-plata_balcon_lavadero_toilette_garage_estrenar_antonini-propiedades_dcp,"venta de departamento en decimo piso al frente, a estrenar. living comedor con ventanales hacia el balcon con pisos de madera. cocina completa con doble mesada, muebles bajo mesada. lavadero separado. toilette.dormitorio con placard. segundo dormitorio a dividir. calefaccion por radiadorbano completo.balcon corrido. posibilidad de cochera semicubierta. u$d 20.000 aviso publicado por pixel inmobiliario (servicio de paginas web para inmobiliarias).",venta depto 2 dorm. a estrenar 7 e/ 36 y 37,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv45KAKirGL3JM8=/trim/198x0/smart/filters:strip_icc()/antoninipropiedades.com/photos/401-1-7%20y%2037%2010A%201.jpg


### Patterns

In [7]:
# Comentar lo que no se deba ejecutar

#pattern = r'(\d+)\s[m]2' # Patron para buscar metros cuadrados (m2)
pattern = r'(\d*\w*)\s*amb'  # Patron para buscar metros ambientes
#pattern = r'(\d*)\s?\b[aA]mbientes\w*'
#pattern = r'(\d*\w*)\s*usd'
#pattern = r'(\d*\w*)\s*habitacion|(\d*\w*)\s?cuarto|(\d*\w*)\s?dorm|(\d*\w*)\s?pieza'

In [8]:
regex = re.compile(pattern, flags = re.IGNORECASE)

### Pruebas de campos

In [9]:
df.iloc[55476]['description']

'departamento en primer piso por escalera, dos plantas. amplias vistas al lago gutierrez y los cerros. pb: hall frio. cocina integrada al living mediante barra con espacio para lavarropas. mueble secarropa. living comedor con salida a amplio balcon. 02 habitaciones, una en suite. toilette c/ducha. pa: suite principal con vestidor y bano con jacuzzi doble y ducha. pisos de madera plastificada y ventanas de pvc. incluye baulera y espacio guardacoche.'

### Funciones

In [10]:
# Busca claves segun pattern en columna dentro de dataFrame 
# Entrada: pattern, columna, df
# Salida: DataFrame

def busca_claves(pattern, columna, df_aux):
    regex = re.compile(pattern, flags = re.IGNORECASE)
    m = pd.DataFrame([regex.findall(n) for n in df_aux[columna].head(100)])
    return m[0]

In [11]:
def existe_clave(key, columna, df_aux):    
    m = []
    for frase in df_aux[columna]:
        if key in frase:
            m.append(True)
        else:
            m.append(False)
    return sum(m)


In [12]:
def quitar_caracteres(column):
    for i in range(len(column)):
        if type(column[i]) == str:
            column[i] = str.lower(unidecode.unidecode(column[i]))

In [ ]:
quitar_caracteres(properatti1["place_name"])
quitar_caracteres(properatti1["place_with_parent_names"])
quitar_caracteres(properatti1["country_name"])
quitar_caracteres(properatti1["state_name"])
quitar_caracteres(properatti1["description"])
quitar_caracteres(properatti1["title"])


In [13]:
existe_clave("$","description",df_a1)


19348

### Concatenar DFs

In [ ]:
pd.concat([df3, df4], axis='columns')

### TESTING Regex

In [ ]:
df1 = busca_claves(pattern,'description',df_a1)
#df1 = busca_claves(pattern,'description',df_a1)
#df2 = busca_claves(pattern,'title',df_a1)

In [ ]:
#prod.apply(lambda x: pd.api.types.infer_dtype(x.values))

In [ ]:
# Dejamos todos los strings en minusculas sin acentos

quitar_caracteres(df1)
df1 = pd.DataFrame(df1)


df1
#df1.rename(index={0:'ambientes'}, inplace=True)


### Rutina para reemplazar literales a numeros

In [14]:
# Armo DF de ambientes

df1 = busca_claves(pattern,'description',df_a1).head(30)
df1

0     2   
1     None
2     2   
3     3   
4     None
5     None
6     None
7     mono
8     dos 
9     None
10    None
11    1   
12    2   
13    3   
14        
15    None
16    4   
17    3   
18    los 
19    t   
20    None
21    None
22    los 
23    t   
24    4   
25    4   
26    None
27    None
28    4   
29    sus 
Name: 0, dtype: object

In [15]:
# Coloco nombre en la columna para luego hacer merge

df1.rename(columns={0:'ambientes'}, inplace=True)

0     2   
1     None
2     2   
3     3   
4     None
5     None
6     None
7     mono
8     dos 
9     None
10    None
11    1   
12    2   
13    3   
14        
15    None
16    4   
17    3   
18    los 
19    t   
20    None
21    None
22    los 
23    t   
24    4   
25    4   
26    None
27    None
28    4   
29    sus 
dtype: object

In [32]:
dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
lista_prop = ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete']

def cambiar_x_nros(df):
    aux1 = pd.DataFrame([x if x not in lista_prop else dic.get(x) for x in df1])
    aux2 = pd.DataFrame([x if x == int else np.nan for x in aux1])
    return aux1

# Le voy a dar otra vuelta para reemplazar los no numericos por NaNs

In [33]:
cambiar_x_nros(df1)

,0
0,2
1,None
2,2
3,3
4,None
5,None
6,None
7,1
8,2
9,None


### Fin rutina 

In [ ]:
#df1['ambientes'] = df1['ambientes'].apply(lambda x: dic.get(x))
#df1

#df1['ambientes'].apply(lambda x: dic.get(x))
#df["Date"] = df["Member"].apply(lambda x: d.get(x))

In [ ]:
# DF para join muchos a muchos

df_conve = pd.DataFrame({'ambientes': ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete'],
           'numero': [1,1,1,2,3,4,5,6,7]})
df_conve


In [ ]:
# Pasaje de cifras literales a nros

df2 = pd.merge(df1,df_conve)
df2

# SIN USO

In [ ]:
#tmp_df = df["cases"].str.extract("([a-z])([0-9]+)([0-9]{2})",expand=True) 
tmp_df = df["description"].str.extract("((\d*)\s?\b[aA]mb\w*)",expand=True) 
tmp_df.dropna(axis=0)

In [ ]:
# TEST: Regex de prueba. Alimenta un DF con el resultado
# CONDICION: No lee NaN's
# NOTA: Me di cuenta que REGEX analiza todo el str y en muchos casos detecta la KEY en varias partes del 
#   str, ergo, al capturar el siguiente dato, lo incluye dentro de una lista y eso lo representa en otra columna
#   Entonces ponerle nombre a una única columna no tendría sentido. Si verificamos con .head(10) no vamos
#   a tener problemas porque justo las 10 primeras filas no tienen mas de una KEY en el str a analizar...

#m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(10)], columns=["ambientes"] )
m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(50)])
#m = pd.DataFrame([regex.findall(n) for n in df_a1['description']])
display(len(m))
#m
m[0]


In [ ]:
# Para pasar una linea a un tipo en particular:

c = "1"
pd.to_numeric(c,int)